In [41]:
from collections import Counter 

import pandas as pd
import numpy as np
import re

In [42]:
df = pd.read_csv("cleaned_train.csv")
# df = pd.read_csv("train.csv")

In [48]:
# df = df[:200]
df.rename(columns={"street":"Street"}, inplace=True)
df = df.replace(np.nan, "")
df

,id,raw_address,POI,Street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",,
2,2,setu siung 119 rt 5 1 13880 cipayung,,siung
3,3,"toko dita, kertosono",toko dita,
4,4,jl. orde baru,,jl. orde baru
...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,


In [76]:
def preprocess(s) :
    s = s.replace("\"", "")
    s = s.replace("\t", " ")
    s = re.sub(r'([,/\\]+)', r' \1 ', s) # 
    s = re.sub(' +', ' ', s) # change multiple spaces into one space
    return s.strip()

df["raw_address"] = df["raw_address"].apply(preprocess)

In [77]:
def create_label(s, poi, street) :
    tokens = s.split(" ")
    n = len(tokens)
    tp = poi.split(" ")
    ts = street.split(" ")
    tags = []
    i = 0
    while i < n :
        if len(tp) > 0 and tokens[i] == tp[0] :
            j = 1
            while j < len(tp) :
                if i+j < n and tokens[i+j] == tp[j]:
                    j += 1
                else :
                    break
            if j == len(tp) :
                tags.append("B-ORGANISATION")
                for k in range(1, j) :
                    tags.append("I-ORGANISATION")
                i += j
            else :
                tags.append("O")
                i += 1
        elif len(ts) > 0 and tokens[i] == ts[0] :
            j = 1
            while j < len(ts) :
                if i+j < n and tokens[i+j] == ts[j] :
                    j += 1
                else :
                    break
            if j == len(ts) :
                tags.append("B-PLACE")
                for k in range(1, j) :
                    tags.append("I-PLACE")
                i += j
            else :
                tags.append("O")
                i += 1                
        else :
            tags.append("O")
            i += 1
        
#         print(tokens)
#         print(tags)

    if tokens[-1] != "." :
        tokens.append(".")
        tags.append("O")
    
    res = ""
#     print(tokens)
#     print(tags)
    for token, tag in zip(tokens, tags) :
        res += "{}\t{}\n".format(token, tag)
    return res

In [78]:
s = "raya samb gede , 299 toko bb kids"
poi = "toko bb kids"
street = "raya samb gede"

res = create_label(s, poi, street)
print(res)

raya	B-PLACE
samb	I-PLACE
gede	I-PLACE
,	O
299	O
toko	B-ORGANISATION
bb	I-ORGANISATION
kids	I-ORGANISATION
.	O



In [79]:
df["Label"] = df.apply(lambda x: create_label(x.raw_address, x.POI, x.Street), axis=1)

In [81]:
def sanity_check() :
    for sentence, labels, poi, street in zip(df["raw_address"], df["Label"], df["POI"], df["Street"]) :
        for l in labels.split("\n") :
            if len(l.split("\t")) > 2 :
                print("Sentence:", sentence)
                print("POI:", poi)
                print("Street:", street)
                print("Bad Label: ", l)
                return
sanity_check()

In [82]:
df

,id,raw_address,POI,Street,Label
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,jl kapuk timur delta sili iii lippo cika,jl\tB-PLACE\nkapuk\tI-PLACE\ntimur\tI-PLACE\nd...
1,1,"aye , jati sampurna",,,"aye\tO\n,\tO\njati\tO\nsampurna\tO\n.\tO\n"
2,2,setu siung 119 rt 5 1 13880 cipayung,,siung,setu\tO\nsiung\tB-PLACE\n119\tO\nrt\tO\n5\tO\n...
3,3,"toko dita , kertosono",toko dita,,"toko\tB-ORGANISATION\ndita\tI-ORGANISATION\n,\..."
4,4,jl. orde baru,,jl. orde baru,jl.\tB-PLACE\norde\tI-PLACE\nbaru\tI-PLACE\n.\...
...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,,jend ahmad yani,jend\tB-PLACE\nahmad\tI-PLACE\nyani\tI-PLACE\n...
299996,299996,"raya cila kko , cilandak timur kel.",,raya cila kko,"raya\tB-PLACE\ncila\tI-PLACE\nkko\tI-PLACE\n,\..."
299997,299997,tanjung gusta jl. yaya 2 no 17,,,tanjung\tO\ngusta\tO\njl.\tO\nyaya\tO\n2\tO\nn...
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,,jalan\tO\ncipadu\tO\njaya\tO\ntaman\tB-ORGANIS...


In [83]:
final = df.sample(frac=1, random_state=0).reset_index(drop=True)
final

,id,raw_address,POI,Street,Label
0,112692,"warung bakso ser bah , 64157",warung bakso,ser bah,warung\tB-ORGANISATION\nbakso\tI-ORGANISATION\...
1,19498,komplek graha teluk jakarta blok.ac 5 no.7 clu...,komplek graha teluk jakarta,,komplek\tB-ORGANISATION\ngraha\tI-ORGANISATION...
2,31689,gedung menara enjiniring - pln enjiniring lant...,menara enjiniring,jl. ciputat raya,gedung\tO\nmenara\tB-ORGANISATION\nenjiniring\...
3,231780,kepuharjo nan 28 lumajang,,,kepuharjo\tO\nnan\tO\n28\tO\nlumajang\tO\n.\tO\n
4,4298,kejayaan 17 4 7 krukut rt 11 1 taman sari,,kejayaan,kejayaan\tB-PLACE\n17\tO\n4\tO\n7\tO\nkrukut\t...
...,...,...,...,...,...
299995,211543,"ger kato stasi maria ratu damai dkb i ,",gereja katolik stasi maria ratu damai dkb i,,ger\tO\nkato\tO\nstasi\tO\nmaria\tO\nratu\tO\n...
299996,86293,tanjung baru arif rah hakim 9,,,tanjung\tO\nbaru\tO\narif\tO\nrah\tO\nhakim\tO...
299997,122579,sumur dewa kar indah 6 selebar,,kar indah 6,sumur\tO\ndewa\tO\nkar\tB-PLACE\nindah\tI-PLAC...
299998,152315,"trun 5 ,",,trun 5,"trun\tB-PLACE\n5\tI-PLACE\n,\tO\n.\tO\n"


In [92]:
n = len(final)
nt = int(0.8 * n)
train = final[:nt+1]
test = final[nt:]

In [93]:
train

,id,raw_address,POI,Street,Label
0,112692,"warung bakso ser bah , 64157",warung bakso,ser bah,warung\tB-ORGANISATION\nbakso\tI-ORGANISATION\...
1,19498,komplek graha teluk jakarta blok.ac 5 no.7 clu...,komplek graha teluk jakarta,,komplek\tB-ORGANISATION\ngraha\tI-ORGANISATION...
2,31689,gedung menara enjiniring - pln enjiniring lant...,menara enjiniring,jl. ciputat raya,gedung\tO\nmenara\tB-ORGANISATION\nenjiniring\...
3,231780,kepuharjo nan 28 lumajang,,,kepuharjo\tO\nnan\tO\n28\tO\nlumajang\tO\n.\tO\n
4,4298,kejayaan 17 4 7 krukut rt 11 1 taman sari,,kejayaan,kejayaan\tB-PLACE\n17\tO\n4\tO\n7\tO\nkrukut\t...
...,...,...,...,...,...
239996,59411,"jalan sisingama , 9",,jalan sisingamangaraja,"jalan\tO\nsisingama\tO\n,\tO\n9\tO\n.\tO\n"
239997,177224,"toko mbak tenik , suta",toko mbak tenik,suta,toko\tB-ORGANISATION\nmbak\tI-ORGANISATION\nte...
239998,137236,"kosar baru ,",,kosar baru,"kosar\tB-PLACE\nbaru\tI-PLACE\n,\tO\n.\tO\n"
239999,9644,"ud budi luhur , raya tuban semanding",ud budi luhur,raya tuban,ud\tB-ORGANISATION\nbudi\tI-ORGANISATION\nluhu...


In [94]:
def save_file(filename, arr) :
    f = open(filename, "w+",  encoding='utf-8')
    for s in arr :
        f.write(s.encode("ascii", "ignore").decode())
        f.write("\n")
    f.close()

In [95]:
ntry = None
# ntry = 1000
if ntry :
    save_file("train_preprocess.txt", train["Label"].values.tolist()[:ntry])
    save_file("valid_preprocess.txt", test["Label"].values.tolist()[:ntry])
else :
    save_file("train_preprocess.txt", train["Label"].values.tolist())
    save_file("valid_preprocess.txt", test["Label"].values.tolist())